## index

In [3]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.ticker import Formatter
import glob
import os

index = pd.read_csv("D:\\work\\project 10 weekly report\\index.csv", encoding="GBK")
index["时间"] = index["时间"].apply(lambda x: int(x.split("/")[0] + x.split("/")[1].zfill(2) + x.split("/")[2].zfill(2)))
index = index.sort_values(by=["代码", "时间"])
display(index.head())
ly = 20191231
llw_e = 20200403
lw_e = 20200410
tw_e = 20200417
index = index[(index["时间"] >= llw_e)|(index["时间"] == ly)]
index.groupby('指数')["时间"].nunique()

,时间,代码,指数,收盘价,成交额,换手率
0,20191231,000001.SH,上证指数,3050.1240,2.270000e+11,0.619515
7,20200306,000001.SH,上证指数,3034.5113,3.770000e+11,1.017185
8,20200309,000001.SH,上证指数,2943.2907,4.380000e+11,1.164404
9,20200310,000001.SH,上证指数,2996.7618,4.250000e+11,1.104503
10,20200311,000001.SH,上证指数,2968.5174,3.790000e+11,0.989757


指数
上证50      11
上证指数      11
中证1000    11
中证500     11
创业板指      11
沪深300     11
深证成指      11
Name: 时间, dtype: int64

In [54]:
df = pd.merge(index[index["时间"] == tw_e][["指数", "收盘价"]], index[index["时间"] == lw_e][["指数", "收盘价"]], on="指数")
df = df.rename(columns={"收盘价_x":"收盘价_twe", "收盘价_y":"收盘价_lwe"})
df = pd.merge(df, index[index["时间"] == llw_e][["指数", "收盘价"]], on="指数")
df = df.rename(columns={"收盘价":"收盘价_llwe"})
df = pd.merge(df, index[index["时间"] == ly][["指数", "收盘价"]], on="指数")
df = df.rename(columns={"收盘价":"收盘价_ly"})
df["本周收益率"] = (df["收盘价_twe"] - df["收盘价_lwe"])/df["收盘价_lwe"]
df["上周收益率"] = (df["收盘价_lwe"] - df["收盘价_llwe"])/df["收盘价_llwe"]
df["今年表现"] = (df["收盘价_twe"] - df["收盘价_ly"])/df["收盘价_ly"]

df = pd.merge(df, index[(index["时间"] > llw_e) & (index["时间"] <= lw_e)].groupby("指数")["换手率"].mean().reset_index(), on="指数")
df = df.rename(columns={"换手率":"上周换手率"})
df = pd.merge(df, index[(index["时间"] > lw_e) & (index["时间"] <= tw_e)].groupby("指数")["换手率"].mean().reset_index(), on="指数")
df = df.rename(columns={"换手率":"本周换手率"})
df = pd.merge(df, index[(index["时间"] > llw_e) & (index["时间"] <= lw_e)].groupby("指数")["成交额"].mean().reset_index(), on="指数")
df = df.rename(columns={"成交额":"上周成交额"})
df = pd.merge(df, index[(index["时间"] > lw_e) & (index["时间"] <= tw_e)].groupby("指数")["成交额"].mean().reset_index(), on="指数")
df = df.rename(columns={"成交额":"本周成交额"})

df["上周成交额"] = df["上周成交额"]/100000000
df["上周成交额"] = df["上周成交额"].apply(lambda x: '%.0f'%(x))
df["本周成交额"] = df["本周成交额"]/100000000
df["本周成交额"] = df["本周成交额"].apply(lambda x: '%.0f'%(x))
df["本周收益率"] = df["本周收益率"].apply(lambda x: '%.2f%%'%(100*x))
df["上周收益率"] = df["上周收益率"].apply(lambda x: '%.2f%%'%(100*x))
df["今年表现"] = df["今年表现"].apply(lambda x: '%.2f%%'%(100*x))
df["上周换手率"] = df["上周换手率"].apply(lambda x: '%.2f%%'%(x))
df["本周换手率"] = df["本周换手率"].apply(lambda x: '%.2f%%'%(x))
df["收盘价_twe"] = df["收盘价_twe"].apply(lambda x: '%.2f'%(x))
df = df.set_index(["指数"])
df = df.reindex(["上证指数", "深证成指", "创业板指", "上证50", "沪深300", "中证500", "中证1000"])
df = df.reset_index()
df = df.rename(columns={"收盘价_twe":"收盘价"})
df = df[["指数", "收盘价", "本周收益率", "上周收益率", "今年表现", "本周换手率", "上周换手率", "本周成交额", "上周成交额"]]
df

,指数,收盘价,本周收益率,上周收益率,今年表现,本周换手率,上周换手率,本周成交额,上周成交额
0,上证指数,2838.49,1.50%,1.18%,-6.94%,0.58%,0.69%,2284,2582
1,深证成指,10527.99,2.23%,1.86%,0.93%,3.76%,4.21%,3846,4245
2,创业板指,2020.77,3.64%,2.27%,12.38%,9.79%,10.98%,1294,1435
3,上证50,2809.08,2.15%,1.51%,-8.30%,0.16%,0.18%,451,450
4,沪深300,3839.49,1.87%,1.51%,-6.28%,0.38%,0.41%,1624,1718
5,中证500,5318.45,2.11%,1.99%,0.96%,1.32%,1.49%,1254,1345
6,中证1000,5710.05,1.29%,2.37%,2.57%,2.07%,2.42%,1564,1768


## index component median

In [71]:
path = r'D:\work\project 10 weekly report\周收益率'
all_files = np.array(glob.glob(os.path.join(path, "*.csv")))
rt = pd.DataFrame()
for i in range(len(all_files)):
    df = pd.read_csv(all_files[i], encoding = "GBK")
    display(all_files[i])
    display(df[df["涨跌幅"].isnull()])
    rt = pd.concat([rt, df["涨跌幅"]], axis=1)
    rt = rt.rename(columns={"涨跌幅": os.path.basename(all_files[i]).split('.')[0]})
rt = rt[["上证指数", "深证成指", "创业板指", "沪深300", "中证500", "中证1000"]]
rt.count(axis=0)

'D:\\work\\project 10 weekly report\\周收益率\\上证50.csv'

,时间,代码,Unnamed: 2,涨跌幅


'D:\\work\\project 10 weekly report\\周收益率\\上证指数.csv'

,时间,代码,Unnamed: 2,涨跌幅
117,2020-04-17,600145.SH,*ST新亿,NaN
680,2020-04-17,600816.SH,安信信托,NaN


'D:\\work\\project 10 weekly report\\周收益率\\中证1000.csv'

...

'D:\\work\\project 10 weekly report\\周收益率\\沪深300.csv'

,时间,代码,Unnamed: 2,涨跌幅
200,2020-04-17,600816.SH,安信信托,NaN


'D:\\work\\project 10 weekly report\\周收益率\\深证成指.csv'

,时间,代码,Unnamed: 2,涨跌幅
424,2020-04-17,300197.SZ,铁汉生态,NaN


上证指数      1548
深证成指       499
创业板指       100
沪深300      299
中证500      499
中证1000     999
dtype: int64

In [74]:
for i in rt.columns:
    print(i)
    print(np.median(rt[i][~rt[i].isnull()]))

上证指数
0.0312920975
深证成指
1.013513514
创业板指
2.6726436785
沪深300
0.7766990290000001
中证500
0.47562425700000005
中证1000
0.158856235


## basis

In [1]:
basis = pd.read_csv("D:\\work\\project 10 weekly report\\basis.csv", encoding="GBK")
jg = pd.read_csv("D:\\work\\project 10 weekly report\\交割日.csv", encoding="GBK")
jg['代码'] = jg['代码'].str[:-4]
jg = jg[jg['最后交易日期'] != '——']
jg['最后交易日期'] = jg['最后交易日期'].apply(lambda x: int(x.split("/")[0] + x.split("/")[1].zfill(2) + x.split("/")[2].zfill(2)))
jg = jg[["代码", "最后交易日期"]]
jg["日期1"] = lw_e
jg["日期2"] = tw_e
for col in ["最后交易日期", "日期1", "日期2"]:
    jg[col] = pd.to_datetime(jg[col].astype(str), format='%Y%m%d')
jg['diff1'] = (jg['最后交易日期'] - jg['日期1']).apply(lambda x: x.days)
jg['diff2'] = (jg['最后交易日期'] - jg['日期2']).apply(lambda x: x.days)
jg

NameError: name 'pd' is not defined

In [129]:
basis["IF2005"] = (basis["IF2005.CFE"] - basis["000300.SH"])/basis["000300.SH"]
basis["IF2006"] = (basis["IF2006.CFE"] - basis["000300.SH"])/basis["000300.SH"]
basis["IF2009"] = (basis["IF2009.CFE"] - basis["000300.SH"])/basis["000300.SH"]
basis["IC2005"] = (basis["IC2005.CFE"] - basis["000905.SH"])/basis["000905.SH"]
basis["IC2006"] = (basis["IC2006.CFE"] - basis["000905.SH"])/basis["000905.SH"]
basis["IC2009"] = (basis["IC2009.CFE"] - basis["000905.SH"])/basis["000905.SH"]
basis = basis[["IF2005", "IF2006", "IF2009", "IC2005", "IC2006", "IC2009","日期"]]
basis = basis[basis["日期"].isin(['2020/4/10', '2020/4/17'])]
basis["日期"] = basis["日期"].apply(lambda x: int(x.split("/")[0] + x.split("/")[1].zfill(2) + x.split("/")[2].zfill(2)))
basis = basis.T
basis = basis.reset_index()
basis = pd.merge(basis, jg, left_on="index", right_on="代码", how="left")
basis["kk"] = basis[0]*365/basis['diff1']
basis["kk1"] = basis[5]*365/basis['diff2']
for col in [0, 5, "kk", "kk1"]:
    basis[col] = basis[col].apply(lambda x: '%.2f%%'%(100*x))
basis = basis.set_index(["index"])
basis = basis.reindex(["IC2005", "IC2006", "IC2009", "IF2005", "IF2006", "IF2009"])
basis = basis.reset_index()
basis = basis[["index", 5, 'kk1', 0, 'kk']]
basis

,index,5,kk1,0,kk
0,IC2005,-1.25%,-16.34%,-1.34%,-13.96%
1,IC2006,-2.67%,-15.45%,-2.60%,-13.55%
2,IC2009,-5.40%,-12.81%,-5.19%,-11.77%
3,IF2005,-1.12%,-14.56%,-0.65%,-6.74%
4,IF2006,-2.16%,-12.51%,-1.56%,-8.16%
5,IF2009,-3.92%,-9.29%,-2.99%,-6.78%


## industry

In [188]:
industry = pd.read_csv("D:\\work\\project 10 weekly report\\industry.csv", encoding="GBK")
industry["时间"] = industry["时间"].apply(lambda x: int(x.split("/")[0] + x.split("/")[1].zfill(2) + x.split("/")[2].zfill(2)))
industry["index"] = industry["index"].str[:-4]
industry

,时间,代码,index,收盘价,成交额
0,20191231,801010.SWI,农林牧渔,3308.100,1.686845e+10
1,20191231,801020.SWI,采掘,2515.258,6.478614e+09
2,20191231,801030.SWI,化工,2703.509,3.390708e+10
3,20191231,801040.SWI,钢铁,2132.800,4.548851e+09
4,20191231,801050.SWI,有色金属,3037.967,2.598167e+10
...,...,...,...,...,...
303,20200413,801890.SWI,机械设备,1136.706,2.400853e+10
304,20200414,801890.SWI,机械设备,1153.451,2.375424e+10
305,20200415,801890.SWI,机械设备,1139.513,2.677350e+10
306,20200416,801890.SWI,机械设备,1140.734,2.335243e+10


In [189]:
ly = 20191231
llw_e = 20200403
lw_e = 20200410
tw_e = 20200417
industry = industry[(industry["时间"] >= llw_e)|(industry["时间"] == ly)]
industry.groupby('index')["时间"].nunique()

index
交通运输    11
休闲服务    11
传媒      11
公用事业    11
农林牧渔    11
化工      11
医药生物    11
商业贸易    11
国防军工    11
家用电器    11
建筑材料    11
建筑装饰    11
房地产     11
有色金属    11
机械设备    11
汽车      11
电子      11
电气设备    11
纺织服装    11
综合      11
计算机     11
轻工制造    11
通信      11
采掘      11
钢铁      11
银行      11
非银金融    11
食品饮料    11
Name: 时间, dtype: int64

In [196]:
df = pd.merge(industry[industry["时间"] == tw_e][["index", "收盘价"]], industry[industry["时间"] == lw_e][["index", "收盘价"]], on="index")
df = df.rename(columns={"收盘价_x":"收盘价_twe", "收盘价_y":"收盘价_lwe"})
df = pd.merge(df, industry[industry["时间"] == llw_e][["index", "收盘价"]], on="index")
df = df.rename(columns={"收盘价":"收盘价_llwe"})
df = pd.merge(df, industry[industry["时间"] == ly][["index", "收盘价"]], on="index")
df = df.rename(columns={"收盘价":"收盘价_ly"})
df["本周收益率"] = (df["收盘价_twe"] - df["收盘价_lwe"])/df["收盘价_lwe"]
df["上周收益率"] = (df["收盘价_lwe"] - df["收盘价_llwe"])/df["收盘价_llwe"]
df["今年表现"] = (df["收盘价_twe"] - df["收盘价_ly"])/df["收盘价_ly"]

df = pd.merge(df, industry[(industry["时间"] > llw_e) & (industry["时间"] <= lw_e)].groupby("index")["成交额"].mean().reset_index(), on="index")
df = df.rename(columns={"成交额":"上周成交额"})
df = pd.merge(df, industry[(industry["时间"] > lw_e) & (industry["时间"] <= tw_e)].groupby("index")["成交额"].mean().reset_index(), on="index")
df = df.rename(columns={"成交额":"本周成交额"})
df = df.sort_values(by="本周收益率", ascending=False)
df["上周成交额"] = df["上周成交额"]/100000000
df["上周成交额"] = df["上周成交额"].apply(lambda x: '%.0f'%(x))
df["本周成交额"] = df["本周成交额"]/100000000
df["本周成交额"] = df["本周成交额"].apply(lambda x: '%.0f'%(x))
# df["本周收益率"] = df["本周收益率"].apply(lambda x: '%.2f%%'%(100*x))
# df["上周收益率"] = df["上周收益率"].apply(lambda x: '%.2f%%'%(100*x))
# df["今年表现"] = df["今年表现"].apply(lambda x: '%.2f%%'%(100*x))
# df["收盘价_twe"] = df["收盘价_twe"].apply(lambda x: '%.2f'%(x))
# df = df[["index", "本周收益率", "上周收益率", "今年表现", "本周成交额", "上周成交额"]]
df

,index,收盘价_twe,收盘价_lwe,收盘价_llwe,收盘价_ly,本周收益率,上周收益率,今年表现,上周成交额,本周成交额
19,电气设备,4637.504,4373.234,4343.165,4354.433,0.060429,0.006923,0.065008,305,329
5,电子,3585.457,3468.923,3511.832,3479.976,0.033594,-0.012218,0.030311,862,764
15,休闲服务,5551.728,5405.705,5112.664,6171.582,0.027013,0.057317,-0.100437,37,31
10,医药生物,9303.149,9085.211,8725.889,7993.154,0.023988,0.041179,0.163890,827,879
21,计算机,5174.619,5060.105,4966.850,4671.440,0.022631,0.018775,0.107714,594,533
2,化工,2596.719,2544.510,2502.749,2703.509,0.020518,0.016686,-0.039501,433,410
18,建筑装饰,2056.561,2017.615,1999.594,2089.110,0.019303,0.009012,-0.015580,115,102
27,机械设备,1151.518,1130.069,1105.111,1172.594,0.018980,0.022584,-0.017974,269,258
17,建筑材料,6786.619,6664.017,6385.482,6318.430,0.018398,0.043620,0.074099,119,110
22,传媒,667.583,655.753,636.154,680.149,0.018040,0.030809,-0.018475,309,284


## futures

In [208]:
futures = pd.read_csv("D:\\work\\project 10 weekly report\\futuresData.csv", encoding="GBK")
futures["date"] = futures["date"].apply(lambda x: int(x.split("/")[0] + x.split("/")[1].zfill(2) + x.split("/")[2].zfill(2)))
futures = futures.rename(columns={'Open':'持仓量','PX_LAST':'收盘价','PX_SETTLE':'结算价','VOLUME':'成交量', 'date':'时间'})
futures = futures[(futures['时间'] >= 20200403) & (futures['时间'] <= 20200417)]
futures['名称'] = np.where(futures['ticker'] == 'AK1 COMB Comdty', '豆一', np.where(
futures['ticker'] == 'AA1 COMB Comdty', '沪铝', np.where(
futures['ticker'] == 'AC1 COMB Comdty', '玉米', np.where(
futures['ticker'] == 'AE1 COMB Comdty', '豆粕', np.where(
futures['ticker'] == 'AK1 COMB Comdty', '豆一', np.where(
futures['ticker'] == 'APW1 COMB Comdty', '苹果', np.where(
futures['ticker'] == 'AUA1 COMB Comdty', '沪金', np.where(
futures['ticker'] == 'BIT1 COMB Comdty', '沥青', np.where(
futures['ticker'] == 'BLC1 COMB Comdty', '胶合板', np.where(
futures['ticker'] == 'BP1 COMB Comdty', '豆二', np.where(
futures['ticker'] == 'CB1 COMB Comdty', '白糖', np.where(
futures['ticker'] == 'CKC1 COMB Comdty', '焦煤', np.where(
futures['ticker'] == 'CU1 COMB Comdty', '沪铜', np.where(
futures['ticker'] == 'DCE1 COMB Comdty', '鸡蛋', np.where(
futures['ticker'] == 'DCS1 COMB Comdty', '玉米淀粉', np.where(
futures['ticker'] == 'DPR1 COMB Comdty', '粳米', np.where(
futures['ticker'] == 'FGL1 COMB Comdty', '玻璃', np.where(
futures['ticker'] == 'FO1 COMB Comdty', '燃油', np.where(
futures['ticker'] == 'FRE1 COMB Comdty', '纤维板', np.where(
futures['ticker'] == 'IMR1 COMB Comdty', '锰硅', np.where(
futures['ticker'] == 'IOE1 COMB Comdty', '铁矿石', np.where(
futures['ticker'] == 'IRE1 COMB Comdty', '硅铁', np.where(
futures['ticker'] == 'IRI1 COMB Comdty', '早籼', np.where(
futures['ticker'] == 'IRL1 COMB Comdty', '晚籼', np.where(
futures['ticker'] == 'JCI1 COMB Comdty', '红枣', np.where(
futures['ticker'] == 'KEE1 COMB Comdty', '焦炭', np.where(
futures['ticker'] == 'KSP1 COMB Comdty', '纸浆', np.where(
futures['ticker'] == 'LGE1 COMB Comdty', '乙二醇', np.where(
futures['ticker'] == 'PAL1 COMB Comdty', '棕榈', np.where(
futures['ticker'] == 'PBL1 COMB Comdty', '沪铅', np.where(
futures['ticker'] == 'PGR1 COMB Comdty', '液化石油气', np.where(
futures['ticker'] == 'POL1 COMB Comdty', '塑料', np.where(
futures['ticker'] == 'PT1 COMB Comdty', 'PTA', np.where(
futures['ticker'] == 'PVC1 COMB Comdty', 'PVC', np.where(
futures['ticker'] == 'PYL1 COMB Comdty', '聚丙烯', np.where(
futures['ticker'] == 'QCY1 COMB Comdty', '棉纱', np.where(
futures['ticker'] == 'RBT1 COMB Comdty', '螺纹钢', np.where(
futures['ticker'] == 'ROC1 COMB Comdty', '热卷', np.where(
futures['ticker'] == 'RT1 COMB Comdty', '橡胶', np.where(
futures['ticker'] == 'SAI1 COMB Comdty', '沪银', np.where(
futures['ticker'] == 'SCP1 COMB Comdty', '原油', np.where(
futures['ticker'] == 'SH1 COMB Comdty', '豆油', np.where(
futures['ticker'] == 'SRB1 COMB Comdty', '20号胶', np.where(
futures['ticker'] == 'SYR1 COMB Comdty', '苯乙烯', np.where(
futures['ticker'] == 'TRC1 COMB Comdty', '郑煤', np.where(
futures['ticker'] == 'URI1 COMB Comdty', '尿素', np.where(
futures['ticker'] == 'VN1 COMB Comdty', '强麦', np.where(
futures['ticker'] == 'VOO1 COMB Comdty', '普麦',  np.where(
futures['ticker'] == 'VV1 COMB Comdty', '郑棉', np.where(
futures['ticker'] == 'WRT1 COMB Comdty', '线材', np.where(
futures['ticker'] == 'XII1 COMB Comdty', '镍', np.where(
futures['ticker'] == 'XOO1 COMB Comdty', '锡', np.where(
futures['ticker'] == 'ZME1 COMB Comdty', '甲醇', np.where(
futures['ticker'] == 'ZNA1 COMB Comdty', '沪锌', np.where(
futures['ticker'] == 'ZRC1 COMB Comdty', '菜籽', np.where(
futures['ticker'] == 'ZRO1 COMB Comdty', '菜油', np.where(
futures['ticker'] == 'ZRR1 COMB Comdty', '菜粕', np.where(
futures['ticker'] == 'ZST1 COMB Comdty', '不锈钢', np.where(
futures['ticker'] == 'ZSY1 COMB Comdty', '纯碱', np.where(
futures['ticker'] == 'ZVB1 COMB Comdty', '粳稻', np.nan))))))))))))))))))))))))))))))))))))))))))))))))))))))))))))
futures = futures[['时间', 'ticker','名称',"持仓量","收盘价",'结算价','成交量']]
futures = futures[~futures['成交量'].isnull()]
futures

,时间,ticker,名称,持仓量,收盘价,结算价,成交量
3,20200407,BLC1 COMB Comdty,胶合板,14.0,167.95,167.95,18.0
4,20200408,BLC1 COMB Comdty,胶合板,31.0,175.30,173.10,49.0
5,20200409,BLC1 COMB Comdty,胶合板,32.0,175.10,175.80,22.0
6,20200410,BLC1 COMB Comdty,胶合板,33.0,175.60,175.90,33.0
7,20200413,BLC1 COMB Comdty,胶合板,32.0,176.10,177.35,3.0
...,...,...,...,...,...,...,...
705,20200413,XII1 COMB Comdty,镍,100998.0,96430.00,96880.00,346151.0
706,20200414,XII1 COMB Comdty,镍,104470.0,97160.00,96750.00,389144.0
707,20200415,XII1 COMB Comdty,镍,109235.0,97240.00,97630.00,337287.0
708,20200416,XII1 COMB Comdty,镍,104866.0,95760.00,96330.00,190089.0


In [209]:
pp = pd.read_csv("D:\\work\\project 10 weekly report\\所有期货合约.csv", encoding="GBK")
pp["日期"] = pp["日期"].apply(lambda x: int(x.split("/")[0] + x.split("/")[1].zfill(2) + x.split("/")[2].zfill(2)))
pp["类型"] = pp["名称"].str[-4:]
pp = pp[pp["类型"] != "主力连续"]
pp["品种"] = pp["名称"].str[:-4]
pp1 = pd.merge(pp[pp["日期"] == lw_e][["类型","收盘价","结算价","品种"]], pp[pp["日期"] == llw_e][["类型","收盘价","品种"]], on=["类型","品种"])
pp1 = pp1.rename(columns={"类型":"类型1", "结算价":"结算价1", "品种":"品种1", "收盘价_x":"收盘价1", "收盘价_y":"收盘价2"})
pp2 = pd.merge(pp[pp["日期"] == tw_e][["类型","收盘价","结算价","品种"]], pp[pp["日期"] == lw_e][["类型","收盘价","品种"]], on=["类型","品种"])
pp2 = pp2.rename(columns={"类型":"类型1", "结算价":"结算价1", "品种":"品种1", "收盘价_x":"收盘价1", "收盘价_y":"收盘价2"})

In [210]:
fut = pd.read_csv("D:\\work\\project 10 weekly report\\stats.csv", encoding="utf-8")
fut = fut[["GEN_FIRST", "VOLATILITY_30D"]]
fut = fut.rename(columns={"VOLATILITY_30D":"30天滚动年化波动率"})

print(len(futures["名称"].unique()))
df = pd.merge(futures[futures["时间"] == tw_e][["名称", "收盘价"]], futures[futures["时间"] == lw_e][["名称", "收盘价"]], on="名称")
print(len(df["名称"].unique()))
df = df.rename(columns={"收盘价_x":"收盘价_twe", "收盘价_y":"收盘价_lwe"})
df = pd.merge(df, futures[futures["时间"] == lw_e][["名称", "结算价"]], on="名称")
df = df.rename(columns={"结算价":"结算价_lwe"})
df = pd.merge(df, futures[futures["时间"] == tw_e][["名称", "结算价"]], on="名称")
df = df.rename(columns={"结算价":"结算价_twe"})
df = pd.merge(df, futures[futures["时间"] == llw_e][["名称", "收盘价"]], on="名称")
print(len(df["名称"].unique()))
df = df.rename(columns={"收盘价":"收盘价_llwe"})
df = pd.merge(df, futures[futures["时间"] == llw_e][["名称", "结算价"]], on="名称")
df = df.rename(columns={"结算价":"结算价_llwe"})

df = pd.merge(df, pp1, left_on=["名称", "收盘价_lwe", "结算价_lwe"], right_on=["品种1", "收盘价1", "结算价1"], how="left")
df["收盘价_llwe"] = df["收盘价2"]
df = pd.merge(df, pp2, left_on=["名称", "收盘价_twe", "结算价_twe"], right_on=["品种1", "收盘价1", "结算价1"], how="left")
df["收盘价_lwe"] = df["收盘价2_y"]
df["本周收益率"] = (df["收盘价_twe"] - df["收盘价_lwe"])/df["收盘价_lwe"]
df["上周收益率"] = (df["收盘价_lwe"] - df["收盘价_llwe"])/df["收盘价_llwe"]

df = pd.merge(df, futures[(futures["时间"] > llw_e) & (futures["时间"] <= lw_e)].groupby("名称")["成交量"].mean().reset_index(), on="名称")
df = df.rename(columns={"成交量":"上周日均成交量(手)"})
df = pd.merge(df, futures[(futures["时间"] > lw_e) & (futures["时间"] <= tw_e)].groupby("名称")["成交量"].mean().reset_index(), on="名称")
df = df.rename(columns={"成交量":"本周日均成交量(手)"})
df = pd.merge(df, futures[futures["时间"] == tw_e][['名称',"持仓量"]], on="名称")
df = df[(df["本周日均成交量(手)"] >= 70000) & (df["上周日均成交量(手)"] >= 70000)].sort_values(by="本周收益率", ascending=False).reset_index(drop=True)
df = pd.merge(df, futures[futures["时间"] == tw_e][["ticker", "名称"]], on="名称")
df = pd.merge(df, fut, left_on="ticker", right_on="GEN_FIRST")

df['本周日均成交量(手)'] = df['本周日均成交量(手)'].apply(lambda x: '%.2f'%(x/10000) + '万')    
df['上周日均成交量(手)'] = df['上周日均成交量(手)'].apply(lambda x: '%.2f'%(x/10000) + '万')    
df['持仓量'] = df['持仓量'].apply(lambda x: '%.2f'%(x/10000) + '万')  
df["本周收益率"] = df["本周收益率"].apply(lambda x: '%.2f%%'%(100*x))
df["上周收益率"] = df["上周收益率"].apply(lambda x: '%.2f%%'%(100*x))
df["收盘价_twe"] = df["收盘价_twe"].apply(lambda x: '%.2f'%(x))

df = df[["名称","收盘价_twe", "本周收益率", "上周收益率", "本周日均成交量(手)", "上周日均成交量(手)", "持仓量", "30天滚动年化波动率"]]
df

59
57
56
      名称  收盘价_twe  收盘价_lwe  结算价_lwe  结算价_twe  收盘价_llwe  结算价_llwe   类型1  \
0     豆一   4689.0   4529.0   4518.0   4658.0    4848.0    4823.0  2009   
8     鸡蛋   3385.0   3408.0   3391.0   3355.0    3113.0    3083.0  2006   
15    硅铁   5590.0   5590.0   5632.0   5560.0    5552.0    5534.0  2009   
16    沪铜  42280.0  41720.0  41180.0  42200.0   39930.0   39730.0  2006   
21    尿素   1547.0   1582.0   1579.0   1547.0    1636.0    1635.0  2009   
22  20号胶   8245.0   8400.0   8295.0   8200.0    7665.0    7645.0  2006   
26    燃油   1687.0   1750.0   1728.0   1668.0    1526.0    1529.0  2009   
27    红枣  10140.0  10030.0  10075.0  10105.0   10010.0   10025.0  2009   
29    沪锌  16130.0  15660.0  15580.0  16110.0   15560.0   15420.0  2006   
39    棉纱  18795.0  19045.0  19085.0  18865.0   18055.0   18005.0  2009   
53    苹果   7963.0   8325.0   8520.0   7963.0    6601.0    6586.0  2010   
54    棕榈   4688.0   4752.0   4780.0   4626.0    4808.0    4834.0  2009   

       收盘价1     结算价1   品种1  

,名称,收盘价_twe,本周收益率,上周收益率,本周日均成交量(手),上周日均成交量(手),持仓量,30天滚动年化波动率
0,液化石油气,3188.00,5.53%,5.08%,59.97万,42.32万,7.40万,NaN
1,沪铝,12410.00,4.77%,2.60%,9.25万,8.19万,13.53万,29.67
2,豆一,4689.00,3.53%,-0.13%,25.14万,15.95万,11.94万,31.22
3,锰硅,7234.00,3.22%,4.47%,18.36万,11.32万,9.66万,33.45
4,沪锌,16130.00,3.00%,1.06%,9.39万,7.95万,9.86万,33.22
5,铁矿石,612.00,2.09%,5.55%,90.63万,96.76万,62.06万,49.05
6,镍,98970.00,1.71%,5.04%,29.93万,43.22万,11.11万,32.20
7,玻璃,1271.00,1.19%,2.78%,36.37万,15.85万,24.94万,28.90
8,玉米,2065.00,1.08%,0.49%,35.64万,28.74万,84.07万,12.61
9,热卷,3235.00,0.81%,5.73%,29.42万,33.10万,43.14万,25.50


In [3]:
import pandas as pd
kk = pd.read_csv(r'C:\Users\win\Desktop\SW1Industry.csv')
kk = kk.sort_values(by="本周收益率", ascending=False)
kk["上周交易额"] = kk["上周交易额"].apply(lambda x: '%.0f'%(x))
kk["本周交易额"] = kk["本周交易额"].apply(lambda x: '%.0f'%(x))
kk["收盘价"] = kk["收盘价"].apply(lambda x: '%.2f'%(x))
kk["本周收益率"] = kk["本周收益率"].apply(lambda x: '%.2f%%'%(100*x))
kk["上周收益率"] = kk["上周收益率"].apply(lambda x: '%.2f%%'%(100*x))
kk["今年表现"] = kk["今年表现"].apply(lambda x: '%.2f%%'%(100*x))
kk

,Unnamed: 0,申万代码,行业名称,收盘价,本周收益率,上周收益率,今年表现,本周交易额,上周交易额
15,15,SW801210,申万休闲服务,7139.04,6.96%,4.24%,15.68%,52,54
22,22,SW801760,申万传媒,746.06,3.63%,5.19%,9.69%,450,420
10,10,SW801150,申万医药生物,10249.66,3.57%,3.15%,28.23%,754,661
14,14,SW801200,申万商业贸易,3640.80,1.99%,4.01%,12.34%,139,158
19,19,SW801730,申万电气设备,4883.05,1.51%,2.72%,12.14%,267,260
21,21,SW801750,申万计算机,5288.97,0.91%,2.72%,13.22%,459,481
17,17,SW801710,申万建筑材料,6875.63,0.70%,-0.33%,8.82%,103,110
3,3,SW801040,申万钢铁,1897.39,0.49%,-0.43%,-11.04%,35,38
8,8,SW801130,申万纺织服装,1828.22,0.36%,4.38%,-4.76%,59,80
26,26,SW801880,申万汽车,3916.39,0.26%,5.99%,0.07%,241,261


In [5]:
df1 = pd.read_csv(r'C:\Users\win\Desktop\SHCOMP.csv', header=0)
df1


,ID,close,date,weekReturn,yclose,week
0,SH600000,10.200,2020-04-17,-0.000979,10.210,202016
1,SH600004,14.190,2020-04-17,0.058166,13.410,202016
2,SH600006,4.040,2020-04-17,-0.017032,4.110,202016
3,SH600007,14.090,2020-04-17,0.045252,13.480,202016
4,SH600008,3.210,2020-04-17,0.032154,3.110,202016
...,...,...,...,...,...,...
1546,SH900952,0.254,2020-04-17,-0.003922,0.255,202016
1547,SH900953,0.381,2020-04-17,-0.005222,0.383,202016
1548,SH900955,0.217,2020-04-17,-0.013636,0.220,202016
1549,SH900956,1.113,2020-04-17,-0.020246,1.136,202016


In [6]:
df2 = pd.read_csv(r'D:\work\project 10 weekly report\周收益率\上证指数.csv', encoding='GBK')
df2["时间"] = df2["时间"].apply(lambda x: int(x.split('-')[0] + x.split('-')[1] + x.split('-')[2]))
df2 = pd.merge(df2[df2["时间"] == 20200410], df2[df2["时间"] == 20200417], on="代码")
df2["weekReturn"] = (df2["收盘价_y"] - df2["收盘价_x"]) / df2["收盘价_x"]
df2

,时间_x,代码,Unnamed: 2_x,收盘价_x,时间_y,Unnamed: 2_y,收盘价_y,weekReturn
0,20200410,600000.SH,浦发银行,10.210,20200417,浦发银行,10.200,-0.000979
1,20200410,600004.SH,白云机场,13.410,20200417,白云机场,14.190,0.058166
2,20200410,600006.SH,东风汽车,4.110,20200417,东风汽车,4.040,-0.017032
3,20200410,600007.SH,中国国贸,13.480,20200417,中国国贸,14.090,0.045252
4,20200410,600008.SH,首创股份,3.110,20200417,首创股份,3.210,0.032154
...,...,...,...,...,...,...,...,...
1546,20200410,900952.SH,锦港B股,0.255,20200417,锦港B股,0.254,-0.003922
1547,20200410,900953.SH,凯马B,0.383,20200417,凯马B,0.381,-0.005222
1548,20200410,900955.SH,海创B股,0.220,20200417,海创B股,0.217,-0.013636
1549,20200410,900956.SH,东贝B股,1.136,20200417,东贝B股,1.113,-0.020246


In [33]:
df2 = pd.read_csv(r'D:\work\project 10 weekly report\周收益率\中证1000.csv', encoding='GBK')
df2["时间"] = df2["时间"].apply(lambda x: int(x.split('-')[0] + x.split('-')[1] + x.split('-')[2]))
# df1["weekReturn"] = df1["weekReturn"].round(6)
df2 = pd.merge(df2[df2["时间"] == 20200410], df2[df2["时间"] == 20200417], on="代码")
df2["涨跌幅"] = (df2["收盘价_y"] - df2["收盘价_x"]) / df2["收盘价_x"]
df2["涨跌幅"] = df2["涨跌幅"].round(6)
df2["代码"] = df2["代码"].str[-2:] + df2["代码"].str[:6]
df2 = df2[(df2["交易状态_x"]!="连续停牌") & (df2["交易状态_y"]!="连续停牌")]
# df = pd.merge(df1, df2, left_on="ID", right_on="代码", how="outer")
# display(df[df["ID"].isnull()])
# display(df[df["代码"].isnull()])
# df[df["weekReturn"] != df["涨跌幅"]]

In [34]:
df2["涨跌幅"].median()

0.0016389999999999998

In [18]:
"SZ002079" in df2["代码"].values

True

In [19]:
df2[df2["代码"] == "SZ002079"]

,时间_x,代码,Unnamed: 2_x,收盘价_x,时间_y,Unnamed: 2_y,收盘价_y,涨跌幅
157,20200410,SZ002079,苏州固锝,10.54,20200417,苏州固锝,10.54,0.0


In [298]:
df2["涨跌幅"].describe()

count    1551.000000
mean        0.006157
std         0.049982
min        -0.179259
25%        -0.016879
50%         0.000227
75%         0.023561
max         0.530208
Name: 涨跌幅, dtype: float64

In [294]:
df2["涨跌幅"].median()

0.000227

In [2]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.ticker import Formatter
import glob
import os
re = pd.read_csv("C:\\Users\\win\\Desktop\\indexFuture.csv")
re["disc"] = re["disc"].apply(lambda x: '%.2f%%'%(100*x))
re["discAn"] = re["discAn"].apply(lambda x: '%.2f%%'%(100*x))
re[re["date"] == '2021-01-08']

,date,contract,disc,discAn
1,2021-01-08,IF2101,0.18%,9.46%
3,2021-01-08,IF2102,0.10%,0.85%
5,2021-01-08,IF2103,0.02%,0.09%
7,2021-01-08,IF2106,-0.92%,-2.08%
9,2021-01-08,IC2101,-0.11%,-5.56%
11,2021-01-08,IC2102,-1.02%,-8.85%
13,2021-01-08,IC2103,-2.05%,-10.70%
15,2021-01-08,IC2106,-4.39%,-9.94%


In [3]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.ticker import Formatter
import glob
import os
re = pd.read_csv("C:\\Users\\win\\Desktop\\SW1Industry.csv", encoding='GBK').sort_values(by="本周收益率", ascending=False)
re["本周收益率"] = re["本周收益率"].apply(lambda x: '%.2f%%'%(100*x))
re["上周收益率"] = re["上周收益率"].apply(lambda x: '%.2f%%'%(100*x))
re["今年表现"] = re["今年表现"].apply(lambda x: '%.2f%%'%(100*x))
re["本周交易额"] = re["本周交易额"].apply(lambda x: '%.f'%(x))
re["上周交易额"] = re["上周交易额"].apply(lambda x: '%.f'%(x))
re

,申万代码,行业名称,收盘价,本周收益率,上周收益率,今年表现,本周交易额,上周交易额
4,SW801050,申万有色金属,4530.85,10.66%,-0.33%,10.66%,735,569
0,SW801010,申万农林牧渔,4285.38,9.51%,2.81%,9.51%,503,316
19,SW801730,申万电气设备,9250.07,9.10%,0.64%,9.10%,910,733
20,SW801740,申万国防军工,1902.14,6.90%,5.63%,6.90%,574,348
7,SW801120,申万食品饮料,30683.96,6.64%,6.81%,6.64%,847,593
2,SW801030,申万化工,3854.73,5.63%,0.70%,5.63%,726,523
5,SW801080,申万电子,4961.47,4.79%,1.58%,4.79%,1279,885
6,SW801110,申万家用电器,10681.27,4.75%,3.87%,4.75%,197,144
3,SW801040,申万钢铁,2328.86,4.09%,-1.13%,4.09%,80,52
17,SW801710,申万建筑材料,8137.62,4.09%,1.07%,4.09%,179,117


In [4]:
kk = pd.read_excel(r'C:\Users\win\Desktop\commodity.xlsx')
kk = kk.iloc[1:, 9:]
kk = kk.dropna(how='all')
kk.columns = kk.iloc[0, :]
kk = kk.iloc[1:, :].reset_index(drop=True)
kk = kk[(kk["Avg_W"] > 70000) & (kk["Avg_W_L1"] > 70000)]
kk = kk.sort_values(by="Ret_W", ascending=False)
kk["Ret_W"] = kk["Ret_W"].apply(lambda x: '%.2f%%'%(x))
kk['Avg_W'] = kk['Avg_W'].apply(lambda x: '%.1f'%(x/10000) + '万')     
kk['OPEN_INTEREST'] = kk['OPEN_INTEREST'].apply(lambda x: '%.1f'%(x/10000) + '万')   
kk["VOLATILITY_30D"] = kk["VOLATILITY_30D"].apply(lambda x: '%.1f%%'%(x))
kk

1,GEN_FIRST,Ret_W,Ret_W_L1,Avg_W,Avg_W_L1,PX_LAST,OPEN_INTEREST,VOLATILITY_30D
10,IRE1 COMB Comdty,15.84%,-2.87728,36.0万,202657,7664,15.4万,45.9%
41,SCP1 COMB Comdty,11.53%,-2.29922,13.3万,124344,336.5,2.7万,45.1%
8,IMR1 COMB Comdty,10.30%,-2.41935,39.6万,219397,7474,15.3万,28.6%
35,CKC1 COMB Comdty,8.78%,-3.20856,19.1万,238150,1772,13.7万,41.7%
43,FO1 COMB Comdty,7.57%,-3.19549,178.3万,1630181,2216,47.5万,40.1%
47,XII1 COMB Comdty,7.21%,-7.11762,82.0万,809376,132490,18.4万,31.9%
22,IOE1 COMB Comdty,7.13%,-12.9751,32.0万,426582,1067,44.7万,54.7%
44,BIT1 COMB Comdty,7.00%,-4.5045,62.5万,742280,2722,41.8万,36.8%
50,RT1 COMB Comdty,6.62%,-7.11661,46.1万,562390,14820,22.0万,36.2%
53,KSP1 COMB Comdty,6.16%,8.17018,40.6万,317940,6100,15.5万,23.2%
